In [90]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from tqdm import tqdm
from scipy.stats import linregress
from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words
import random
from sklearn.metrics import confusion_matrix 
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /Users/luckywang/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [117]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [118]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [119]:
legit_tickers = ticker_library["ticker"]

In [120]:
df_esg_score = df_esg_score[df_esg_score["Company"].isin(legit_tickers)]

In [121]:
len(df_esg_score)

501

## Random

In [15]:
# TODO: Modify here for different sectors and score_types
# set the sector and ESG type for the analysis
sector = "Financial Services"
# sector = "Energy"

score_type = "governanceScore"
# score_type = "environmentScore"
# score_type = "socialScore"

In [98]:
#training set
train_bad=random.sample(list(bad_companies), int(len(bad_companies)*0.7))
train_good=random.sample(list(good_companies), int(len(good_companies)*0.7))

#validation set
validate_bad = [i for i in bad_companies if i not in train_bad]
validate_good = [i for i in good_companies if i not in train_good]

# validate_good = pd.DataFrame(validate_good) 
# validate_bad = pd.DataFrame(validate_bad) 

In [99]:
validation = pd.DataFrame({'good':validate_good, 'bad':validate_bad})

In [100]:
#save the list of good and bad companies
validation_path = os.path.join("data", "validation_data")
if not os.path.isdir(validation_path):
    os.mkdir(validation_path)

validation.to_csv("data/validation_data/{}_{}_{}.csv".format(sector[:8], score_type[:3], alpha))

In [58]:
print(len(train_good))

20


In [59]:
print(len(validate_good))

9


In [60]:
# esgs[esgs[score_type] > upper_score]

In [61]:
#training data bad companies scores
train_bad_scores=pd.DataFrame()
for i in train_bad:
    df_bad=esgs[esgs['Company'] == i]
    train_bad_scores=train_bad_scores.append(df_bad)

In [62]:
#training data good companies scores
train_good_scores=pd.DataFrame()

for i in train_good:
    df_good=esgs[esgs['Company'] == i]
    train_good_scores =train_good_scores.append(df_good)

In [63]:
# bad_companies_score = esgs[esgs[score_type] > upper_score][score_type].values
# good_companies_score = esgs[esgs[score_type] < lower_score][score_type].values

good_companies_score_training=train_good_scores[score_type]
bad_companies_score_training=train_bad_scores[score_type]

In [64]:
avg_bad = np.mean(bad_companies_score_training)
avg_good = np.mean(good_companies_score_training)
print(avg_bad, avg_good)

11.138499999999999 6.109999999999999


In [143]:
print(upper_score, lower_score)

10.23 10.23


## Utils

In [144]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))
good_cik = []
bad_cik = []
for ticker in train_good:    
    try:
        # for a given ticker, find its cik number through th ticker library
        good_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        good_cik.append('')

for ticker in train_bad:    
    try:
        # for a given ticker, find its cik number through th ticker library
        bad_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        bad_cik.append('')

C:\Users\Mohammed Al Harmoudi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
ret_good = get_texts(good_cik, train_good)

In [ ]:
ret_bad = get_texts(bad_cik, train_bad)

In [147]:
good_docs = ret_good["docs"]
bad_docs = ret_bad["docs"]

In [148]:
# TODO: Modify here for different ngram range
n_min = 2
n_max = 3
cv = CountVectorizer(max_df=0.7, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
word_count_vector = cv.fit_transform(good_docs + bad_docs)

C:\Users\Mohammed Al Harmoudi\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [149]:
feature_names = cv.get_feature_names()

In [150]:
count_feature = word_count_vector.toarray().sum(axis=0)

In [151]:
feature_names = cv.get_feature_names()

In [152]:
d = {"word": [], "good_score": [], "bad_score": [], "good_score_all": []
    , "bad_score_all": [], "count": [], "good_nums": [], "bad_nums": []}

for feature_idx, word in enumerate(feature_names):
    good_sum = bad_sum = good_num = bad_num = 0

    for i, doc_set in enumerate(good_docs):
        if word in doc_set:
            good_num += 1
            good_sum += good_companies_score[i]
    for i, doc_set in enumerate(bad_docs):
        if word in doc_set:
            bad_num += 1
            bad_sum += bad_companies_score[i]
    
    # print("word: {}".format(word))
    d["word"].append(word) 
    
    if good_num:
        d["good_score"].append(good_sum / good_num)
    else:
        d["good_score"].append(0)
    if bad_num:
        d["bad_score"].append(bad_sum / bad_num)
    else:
        d["bad_score"].append(0)

    d["good_score_all"].append(good_sum / len(good_docs))
    d["bad_score_all"].append(bad_sum / len(bad_docs))

    d["count"].append(count_feature[feature_idx])
    d["good_nums"].append(good_num)
    d["bad_nums"].append(bad_num)


In [153]:
df = pd.DataFrame(data=d)


In [154]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums
0,accident year,0.730000,7.668571,0.066364,2.440000,2329,2,7
1,accompanying note,1.002308,7.452308,0.592273,4.403636,847,13,13
2,acquisition cost,0.867273,7.795625,0.433636,5.669545,1249,11,16
3,adjusted earnings,1.016667,6.565000,0.138636,0.596818,918,3,2
4,adjusted operating,1.042857,5.356000,0.331818,1.217273,1454,7,5
5,adjustment expense,0.925000,7.531111,0.168182,3.080909,1133,4,9
6,aflac japan,0.000000,5.870000,0.000000,0.266818,732,0,1
7,allowance loan,0.835833,8.014615,0.455909,4.735909,951,12,13
8,amount except,0.616000,9.350000,0.140000,2.125000,779,5,5
9,available sale,0.760000,7.723077,0.345455,4.563636,871,10,13


In [155]:
df["diff"] = abs(df["good_nums"] - df["bad_nums"])

In [156]:
df = df.sort_values("diff", ascending=False)#.head(60)

In [157]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums,diff
86,insurance subsidiary,0.925000,7.593846,0.168182,4.487273,1246,4,13,9
130,net interest income,0.798667,7.905000,0.544545,2.155909,2101,15,6,9
98,line business,0.931000,7.474444,0.423182,6.115455,1069,10,18,8
135,net revenue,0.900625,7.540000,0.655000,2.741818,867,16,8,8
116,loss ratio,0.955000,7.850833,0.173636,4.282273,846,4,12,8
127,mortgage loan,0.791000,8.332222,0.359545,6.817273,4022,10,18,8
76,future policy,0.925000,7.480000,0.168182,4.080000,1214,4,12,8
150,policy benefit,0.925000,7.610833,0.168182,4.151364,1121,4,12,8
85,insurance product,0.880000,7.723125,0.360000,5.616818,1022,9,16,7
91,invested asset,0.816000,8.008333,0.185455,4.368182,856,5,12,7


In [158]:
goodvbad_path = os.path.join("data", "training_goodvbad")
if not os.path.isdir(goodvbad_path):
    os.mkdir(goodvbad_path)

In [159]:
df.round(2).to_csv("data/training_goodvbad/{}_{}_{}_n{}-{}.csv".format(sector[:8], score_type[:3], alpha, n_min, n_max))

## Validation

In [4]:
#trainig data scores

#Technology_envirnomental
data_tech = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\training_goodvbad\Technolo_env_0.5_n2-3.csv'

#consumer cyclical_social
data_consumer = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\training_goodvbad\Consumer_soc_0.5_n2-3.csv'

#financial services_governance
data_financials = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\training_goodvbad\Financia_gov_0.5_n2-3.csv'

In [5]:
df_tech=pd.read_csv(data_tech).iloc[:,1:]
df_consumer=pd.read_csv(data_consumer).iloc[:,1:]
df_financials=pd.read_csv(data_financials).iloc[:,1:]

In [ ]:
df_tech.isGood.value_counts()

In [18]:
df_consumer.isGood.value_counts()

 1    91
 0    79
-1    30
Name: isGood, dtype: int64

In [7]:
#load validation companies

#consumer cyclical_social
validation_cyclical_data = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\validation_data\Consumer_soc_0.5.csv'

#Technology_envirnomental
validation_tech_data = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\validation_data\Technolo_env_0.5.csv'

#financial services_governance
validation_financials_data = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\validation_data\Financia_gov_0.5.csv'


In [8]:
validation_cyc=pd.read_csv(validation_cyclical_data, usecols=[1,2])
validation_tech=pd.read_csv(validation_tech_data, usecols=[1,2])
validation_financials=pd.read_csv(validation_financials_data, usecols=[1,2])

## Functions

In [74]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

def get_cik(ticker):
    """ Get the cik for the ticker specified by the input argument 
    Input:
        ticker(str): ticker of the company e.g. "FB"
    """
    return ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:]
    
def ngrams(s, n):
    """ Get all the n-gram for input texts s
    Input:
        s (str): A string of texts with each word separated by a whitespace
        n (int): n-gram to extract
    Return:
        [str]: A list of string in the following format ([['a', 'b'], ['b', 'c'], ['c', 'd']])
    """
    
    s = s.split(' ')
    output = []
    for i in range(len(s) - n + 1):
        output.append(s[i:i+n])

    return output

def get_count(doc, df_dict):
    """ Count the number of good and bad words occurred in the document
    Input:
        doc (str): A string with all the words in the documents
        df_dict (pd.DataFrame): A DataFrame with word and isGood column, generated by previous section
        n_min, n_max (int): specify the ngram range used to generate the dictionary, should be consistent with how df_dict is generated
    Return:
        (dict): A dictionary with value good_count and bad_count
    """
    grams = doc.split()
    good_count = bad_count = 0
    
    for g in grams:
        if g in df_dict["word"].values:
            val = df_dict[df_dict["word"] == g]["isGood"].values
            if val == 1:
                good_count += 1
            elif val == -1:
                bad_count += 1

    return {"good_count": good_count, "bad_count": bad_count}
    
def validation(df_topk, val_tickers, dict_threshold):
    """ Perform the validation step
    The validation rationale: Companies whose score are in upper 50% group are considered "bad" companies and the corresponding val_true = 1; 0 otherwise (in lower 50% group, which is considered a good company)
    Input:
        df_topk (pd.DataFrame): containes the sector specific dict
        val_tickers (list): A list of tickers to be validated
        dict_threshold (int): A gram is considered as a good word if its good_nums - bad_nums > threshold
    """
    diff = df_topk["good_nums"] - df_topk["bad_nums"]
    df_topk["isGood"] = diff.apply(lambda x: 1 if x > dict_threshold else (
        -1 if x < -dict_threshold else 0))

    # 1 if good_nums - bad_nums > threshold; -1 if good_nums - bad_nums < -threshold; 0 otherwise
    val_ciks = [get_cik(ticker) for ticker in val_tickers]
    
    ret_texts = get_texts(val_ciks, val_tickers) 

    val_pred = []
    for doc in tqdm(ret_texts["docs"]):
        ret = get_count(doc, df_topk[["word", "isGood"]], 2, 3)

        if ret["good_count"] - ret["bad_count"] > 0:
            val_pred.append(1)
        else:
            val_pred.append(0)
    
    print("val_pred: {}".format(val_pred))
    
    return val_pred


/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# financials sector, governance
val_tickers_financials = (pd.concat([validation_financials['good'], validation_financials['bad']])).tolist()

# TODO: Update to the actual tickers to test
# val_tickers = ["FB", "GOOG"] # should be a list of tickers you split into test set

val_ciks = [get_cik(ticker) for ticker in val_tickers_financials]

val_pred = validation(df_financials, val_tickers_financials, 2)   # df is the dictionary with good_num, bad_num, diff

# val_true (list): A list of true labels for each companies (1 for good company, 0 for bad)
val_true = np.concatenate([([i]*len(validation_financials['good'])) for i in [1,0]], axis=0).tolist()
# val_true = [1, 1]

cm = confusion_matrix(val_true, val_pred)
prec = cm[1][1]/(cm[1][1]+cm[1][0])
rec = cm[1][1]/(cm[1][1]+cm[0][1])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

print("precision: \n{}".format(prec))
print("recall: \n{}".format(rec))
print("accuracy: \n{}".format(acc))
print("Confusion Matrix: \n{}".format(cm))

20it [00:01, 14.56it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:37<00:00, 13.88s/it]

val_pred: [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
precision: 
0.7
recall: 
0.875
accuracy: 
0.8
Confusion Matrix: 
[[9 1]
 [3 7]]


In [ ]:
df_financials.isGood.value_counts()

In [26]:
# 1 is numb of good words
# -1 is numb of bad words
df_financials.isGood.value_counts()

 0    106
-1     63
 1     31
Name: isGood, dtype: int64

In [ ]:
# Consumer cyclical sector, social
val_tickers_cyc = (pd.concat([validation_cyc['good'], validation_cyc['bad']])).tolist()

# TODO: Update to the actual tickers to test
# val_tickers = ["FB", "GOOG"] # should be a list of tickers you split into test set

val_ciks = [get_cik(ticker) for ticker in val_tickers_cyc]

val_pred = validation(df_consumer, val_tickers_cyc, 2)   # df is the dictionary with good_num, bad_num, diff

# val_true (list): A list of true labels for each companies (1 for good company, 0 for bad)
val_true = np.concatenate([([i]*len(validation_cyc['good'])) for i in [1,0]], axis=0).tolist()
# val_true = [1, 1]
cm = confusion_matrix(val_true, val_pred)
prec = cm[1][1]/(cm[1][1]+cm[1][0])
rec = cm[1][1]/(cm[1][1]+cm[0][1])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

print("precision: \n{}".format(prec))
print("recall: \n{}".format(rec))
print("accuracy: \n{}".format(acc))
print("Confusion Matrix: \n{}".format(cm))

## Appendix

In [233]:
# To remove all the previous files earlier than 2020
for cik in tqdm(bad_cik):
    tenk_path = os.path.join("data", "10k", cik, "rawtext")
    all_raws = os.listdir(tenk_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenk_path, filename))
    
    tenq_path = os.path.join("data", "10q", cik, "rawtext")
    all_raws = os.listdir(tenq_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenq_path, filename))


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 440.37it/s]


In [234]:
df = pd.read_csv("energy_good_vs_bad_uni_bi_tri.csv", index_col=0)

FileNotFoundError: [Errno 2] File energy_good_vs_bad_uni_bi_tri.csv does not exist: 'energy_good_vs_bad_uni_bi_tri.csv'

## Model validation for approach 1 (tfidf)

In [75]:
def get_ciks(tickers):
    ciks = []

    for ticker in tickers:
        ciks.append(get_cik(ticker))

    return ciks

In [108]:
def train_dict_tfidf(good_ticker, bad_ticker, sector, score_type, rerun=False):
    path = os.path.join("data", "tfidf_scores", "train", "{}_{}.csv".format(sector, score_type))
    if not rerun and os.path.exists(path):
        df_tfidf = pd.read_csv(path, index_col=0)
    else:
        print("Train {} {}".format(sector, score_type))
        tickers = good_ticker + bad_ticker
        ciks = get_ciks(tickers)

        esgs = df_esg_score[df_esg_score["Company"].isin(tickers)][["Company", "socialScore", "governanceScore", "environmentScore"]]

        for t in not_listed:
            if t in good_ticker or t in bad_ticker:
                esgs = esgs.drop(esgs[esgs["Company"] == t].index)

        ret = get_texts(ciks, tickers)
        docs = ret["docs"]

        cv = CountVectorizer(max_df=0.8, stop_words=stop_words, max_features=1000)
        word_count_vector = cv.fit_transform(docs)

        tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
        tfidf_transformer.fit(word_count_vector)

        feature_names = cv.get_feature_names()

        df_doc_word = pd.DataFrame(columns=feature_names, index=tickers)

        for i, ticker in tqdm(enumerate(tickers)):
            tf_idf_vector = tfidf_transformer.transform(cv.transform([docs[i]]))
            
            coo_matrix = tf_idf_vector.tocoo()
            # coo_matrix: A sparse matrix in which coo_matrix.col stores word_idx, coo_matrix.data stores tfidf score
            
            tuples = zip(coo_matrix.col, coo_matrix.data)
            for word_idx, tfidf in tuples:
                df_doc_word.at[ticker, feature_names[word_idx]] = tfidf

        df_doc_word = df_doc_word.iloc[:, df_doc_word.columns.isin(words.words())]
        df_doc_word = df_doc_word.fillna(0)

        feature_names = [name for name in feature_names if name in words.words()]

        df_tfidf = pd.DataFrame(columns=["{}_beta".format(score_type)], index=feature_names)

        # for typ in ["social", "governance", "environment"]:
        score = esgs[score_type]
        slopes = []
        
        for word in feature_names:
            tfidfs = df_doc_word[word].values.astype(float)
            slope, intercept, *_ = linregress(tfidfs, score)
            slopes.append(slope)
        df_tfidf["{}_beta".format(score_type)] = slopes

        cols = df_tfidf.columns
        alpha = 0.3

        for col in cols: 
            betas = df_tfidf[col]
            score_type = col.split('_')[0]
            
            upper_score = np.quantile(betas, 1 - alpha)
            lower_score = np.quantile(betas, alpha)
            
            is_good = np.where(betas < lower_score, 1, 0) + np.where(betas > upper_score, -1, 0)
            
            df_tfidf["isGood"] = is_good

        df_tfidf.to_csv(path)
    return df_tfidf

In [103]:
def validation_tfidf(df_tfidf, val_tickers):
    """ Perform the validation step
    The validation rationale: Companies whose score are in upper 50% group are considered "bad" companies and the corresponding val_true = 1; 0 otherwise (in lower 50% group, which is considered a good company)
    Input:
        df_topk (pd.DataFrame): containes the sector specific dict
        val_tickers (list): A list of tickers to be validated
    """
    df_tfidf["word"] = df_tfidf.index
    
    # 1 if good_nums - bad_nums > threshold; -1 if good_nums - bad_nums < -threshold; 0 otherwise
    val_ciks = [get_cik(ticker) for ticker in val_tickers]
    
    ret_texts = get_texts(val_ciks, val_tickers)

    val_pred = []
    for i, doc in tqdm(enumerate(ret_texts["docs"])):
        ret = get_count(doc, df_tfidf[["word", "isGood"]])
        print(val_tickers[i])
        print(ret)

        if ret["good_count"] - ret["bad_count"] > 0:
            val_pred.append(1)
        else:
            val_pred.append(0)
    
    print("val_pred: {}".format(val_pred))
    
    return val_pred

In [104]:
def get_report(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    prec = cm[1][1]/(cm[1][1]+cm[1][0])
    rec = cm[1][1]/(cm[1][1]+cm[0][1])
    acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

    print("precision: \n{}".format(prec))
    print("recall: \n{}".format(rec))
    print("accuracy: \n{}".format(acc))
    print("Confusion Matrix: \n{}".format(cm))

    return {"cm": cm, "precision": prec, "recall": rec, "accuracy": acc}


In [ ]:
df_val_precision = pd.DataFrame(index=sectors, columns=score_types)
df_val_recall = pd.DataFrame(index=sectors, columns=score_types)
df_val_accuracy = pd.DataFrame(index=sectors, columns=score_types)
cms = []

In [124]:
# sectors = ['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
#        'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
#        'Financial Services', 'Communication Services', 'Real Estate']
sectors = ['Technology', 'Financial Services']
# sectors = ["Energy"]
score_types = ["governanceScore", "environmentScore", "socialScore"]
not_listed = ["BBWI"]

In [125]:
pair = (('Technology', "socialScore"), ('Technology', "governanceScore"), ('Financial Services', "governanceScore"), ('Financial Services', "socialScore"))

In [126]:
for sector in sectors:
    for score_type in score_types:
        if (sector, score_type) not in pair:
            continue
        print("{} {}".format(sector, score_type))
        esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]
        score = esgs[score_type]
        alpha = 0.3
        upper_score = np.quantile(score, 1 - alpha)
        lower_score = np.quantile(score, alpha)

        good_companies = list(esgs[esgs[score_type] < lower_score]["Company"].values)
        bad_companies = list(esgs[esgs[score_type] > upper_score]["Company"].values)
                
        #training set
        train_good = random.sample(list(good_companies), int(len(good_companies)*0.7))
        train_bad = random.sample(list(bad_companies), int(len(bad_companies)*0.7))

        df_tfidf = train_dict_tfidf(train_good, train_bad, sector, score_type, rerun=False)
        
        #validation set
        validate_good = [ticker for ticker in good_companies if ticker not in train_good]
        validate_bad = [ticker for ticker in bad_companies if ticker not in train_bad]

        val_tickers = validate_good + validate_bad
        
        val_pred = validation_tfidf(df_tfidf, val_tickers)   # df is the dictionary with good_num, bad_num, diff
        val_true = [1] * len(validate_good) + [0] * len(validate_bad)

        val_performance = get_report(val_true, val_pred)
        df_val_precision.at[sector, score_type] = val_performance["precision"]
        df_val_recall.at[sector, score_type] = val_performance["recall"]
        df_val_accuracy.at[sector, score_type] = val_performance["accuracy"]
        cms.append(val_performance["cm"])

Technology governanceScore
Train Technology governanceScore


26it [00:01, 20.42it/s]
26it [00:02, 12.45it/s]


3.2781869018412046
-20.26052969246467
              governanceScore_beta  isGood
absence                  23.371701      -1
absolute                  2.531386       0
acceleration            -14.955977       0
according                -8.046798       0
accrue                   -9.275016       0
accurate                -31.684920       1
achievement              36.636676      -1
acting                    3.153371       0
actuarial               -10.684804       0
adapt                    -2.811467       0
adequacy                -12.426138       0
adjacent                -13.577317       0
adopt                    17.860144      -1
advertising              19.208867      -1
affiliate               -29.481376       1
affirmative             -15.413332       0
age                      -6.945850       0
agent                     7.529615      -1
agree                   -14.952437       0
ai                       14.243032      -1
aid                      -5.013420       0
allegation      

12it [00:00, 19.45it/s]
1it [00:06,  6.34s/it]

NXPI
{'good_count': 1118, 'bad_count': 878}


...

GS
{'good_count': 3522, 'bad_count': 2369}


11it [02:23, 15.23s/it]

MS
{'good_count': 1708, 'bad_count': 1219}


12it [02:34, 12.85s/it]

STT
{'good_count': 2785, 'bad_count': 1581}
val_pred: [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1]
precision: 
0.0
recall: 
0.0
accuracy: 
0.08333333333333333
Confusion Matrix: 
[[1 5]
 [6 0]]


In [127]:
df_val_recall

,governanceScore,environmentScore,socialScore
Consumer Cyclical,0.5,0.25,0.444444
Industrials,0.4,NaN,NaN
Utilities,0.0,0.5,NaN
Technology,0.5,NaN,0.25
Financial Services,0.25,NaN,0.0


In [128]:
df_val_accuracy

,governanceScore,environmentScore,socialScore
Consumer Cyclical,0.5,0.333333,0.416667
Industrials,0.416667,NaN,NaN
Utilities,0.0,0.5,NaN
Technology,0.5,NaN,0.333333
Financial Services,0.166667,NaN,0.083333


In [129]:
df_val_precision

,governanceScore,environmentScore,socialScore
Consumer Cyclical,0.333333,0.166667,0.666667
Industrials,0.333333,NaN,NaN
Utilities,0.0,0.666667,NaN
Technology,0.5,NaN,0.166667
Financial Services,0.333333,NaN,0.0


In [130]:
cms

[array([[4, 2],
        [4, 2]]),
 array([[3, 3],
        [5, 1]]),
 array([[1, 5],
        [2, 4]]),
 array([[1, 5],
        [3, 3]]),
 array([[3, 3],
        [4, 2]]),
 array([[0, 3],
        [3, 0]]),
 array([[1, 2],
        [1, 2]]),
 array([[3, 3],
        [3, 3]]),
 array([[3, 3],
        [5, 1]]),
 array([[0, 6],
        [4, 2]]),
 array([[1, 5],
        [6, 0]])]

In [17]:
df_val_accuracy

,governanceScore,environmentScore,socialScore
Energy,0.5,0.5,0.5


In [109]:
sector = 'Consumer Cyclical'
score_types = "environmentScore"

esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]
score = esgs[score_type]
alpha = 0.3
upper_score = np.quantile(score, 1 - alpha)
lower_score = np.quantile(score, alpha)

bad_companies = list(esgs[esgs[score_type] > upper_score]["Company"].values)
good_companies = list(esgs[esgs[score_type] < lower_score]["Company"].values)

for t in not_listed:
    if t in bad_companies:
        bad_companies.remove(t)
    if t in good_companies:
        good_companies.remove(t)
        
#training set
train_good = random.sample(list(good_companies), int(len(good_companies)*0.7))
train_bad = random.sample(list(bad_companies), int(len(bad_companies)*0.7))

df_tfidf = train_dict_tfidf(train_good, train_bad, sector, score_type, rerun=False)

#validation set
validate_good = [ticker for ticker in good_companies if ticker not in train_good]
validate_bad = [ticker for ticker in bad_companies if ticker not in train_bad]

val_tickers = validate_good + validate_bad

val_pred = validation_tfidf(df_tfidf, val_tickers)   # df is the dictionary with good_num, bad_num, diff
val_true = [1] * len(validate_good) + [0] * len(validate_bad)

# val_performance = get_report(val_true, val_pred)

Train Consumer Cyclical socialScore


24it [00:00, 27.44it/s]
/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
24it [00:02,  9.87it/s]


79.06955651714321
-53.54111784783206
             socialScore_beta  isGood
accept              34.720252       0
accessory          -39.541716       0
accord            -172.654997       1
achievement        127.554267      -1
actuarial          439.797658      -1
add                218.156139      -1
adjust             280.132074      -1
adopt               83.043171      -1
advance            -29.159821       0
advanced            37.672048       0
advertising         77.588309       0
affiliate           -1.181519       0
age                 14.487396       0
air                 23.887095       0
aluminum            13.189829       0
amend             -689.686912       1
amort              -98.618887       1
analyst             16.867392       0
apparel              9.501550       0
appear             189.283871      -1
approval           489.704555      -1
arise              356.007643      -1
asp                  4.514310       0
assert             504.370988      -1
assess       

12it [00:00, 37.69it/s]
1it [00:03,  3.38s/it]

FBHS
{'good_count': 209, 'bad_count': 817}


...

EXPE
{'good_count': 2924, 'bad_count': 2172}


11it [00:46,  4.60s/it]

MAR
{'good_count': 2637, 'bad_count': 1791}


12it [00:48,  4.06s/it]

ROL
{'good_count': 255, 'bad_count': 766}
val_pred: [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]


In [110]:
get_report(val_true, val_pred)

precision: 
0.16666666666666666
recall: 
0.3333333333333333
accuracy: 
0.4166666666666667
Confusion Matrix: 
[[4 2]
 [5 1]]


{'cm': array([[4, 2],
        [5, 1]]),
 'precision': 0.16666666666666666,
 'recall': 0.3333333333333333,
 'accuracy': 0.4166666666666667}

In [ ]:
# Consumer cyclical sector, social
val_tickers_cyc = (pd.concat([validation_cyc['good'], validation_cyc['bad']])).tolist()

# TODO: Update to the actual tickers to test
# val_tickers = ["FB", "GOOG"] # should be a list of tickers you split into test set

val_ciks = [get_cik(ticker) for ticker in val_tickers_cyc]

val_pred = validation(df_consumer, val_tickers_cyc, 2)   # df is the dictionary with good_num, bad_num, diff

# val_true (list): A list of true labels for each companies (1 for good company, 0 for bad)
val_true = np.concatenate([([i]*len(validation_cyc['good'])) for i in [1,0]], axis=0).tolist()
# val_true = [1, 1]
cm = confusion_matrix(val_true, val_pred)
prec = cm[1][1]/(cm[1][1]+cm[1][0])
rec = cm[1][1]/(cm[1][1]+cm[0][1])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

print("precision: \n{}".format(prec))
print("recall: \n{}".format(rec))
print("accuracy: \n{}".format(acc))
print("Confusion Matrix: \n{}".format(cm))